In [1]:
import spacy
from spacy import displacy
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher
from my_nlp_lib import spacy_utilities as su
from my_nlp_lib import text_utilities as tu
su.print_versions()

Python version : 3.8.5 (default, Jul 28 2020, 12:59:40) 
[GCC 9.3.0]
Spacy version  : 3.0.1     
Current dir : /home/cgil/PycharmProjects


## First let us get the first chapter of Victor Hugo "Les Misérables"

In [2]:
first_section_chapters = tu.get_chapters_dic_of_first_section_of_french_book()
first_section_chapter_1 = first_section_chapters['01) Chapitre I']
my_offset = 5
print(f"## OK : let's print {my_offset} first and last lines of the first chapter of the section :")
tu.print_head_and_tail(first_section_chapter_1, offset=my_offset)

## OK : let's print 5 first and last lines of the first chapter of the section :
### 5 FIRST LINES : ###

Monsieur Myriel


En 1815, M. Charles-François-Bienvenu Myriel était évêque de Digne.
########################
### 5 LAST  LINES : ###
L'installation terminée, la ville attendit son évêque à l'oeuvre.




########################


## let's now use spacy to analyse the first chapter

In [3]:
# load the normal french model
nlp = spacy.load('fr_core_news_sm')
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7f9d258e24a0>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x7f9d26fb64a0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7f9d26ff4f40>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f9d26ff4b80>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7f9d26ba5cc0>),
 ('lemmatizer', <spacy.lang.fr.lemmatizer.FrenchLemmatizer at 0x7f9d26e7e7c0>)]

In [4]:
chapter_1 = " ".join(first_section_chapter_1)
chapter_1 = chapter_1.replace('\n\n', '\n')
# print(chapter_1)

In [5]:
doc = nlp(chapter_1)

### let's list the named entities found by spacy

In [6]:
for e in doc.ents:
    print(f'{e.text:{25}} ({e.start:{5}},{e.end:{5}}) {e.label_:{10}} {spacy.explain(e.label_):{40}}')

Charles                   (    8,    9) PER        Named person or family.                 
François                  (   10,   11) PER        Named person or family.                 
Bienvenu Myriel           (   12,   14) PER        Named person or family.                 
Digne                     (   17,   18) LOC        Non-GPE locations, mountain ranges, bodies of water
Digne                     (   33,   34) LOC        Non-GPE locations, mountain ranges, bodies of water
Vrai                      (   98,   99) LOC        Non-GPE locations, mountain ranges, bodies of water
Myriel                    (  128,  129) PER        Named person or family.                 
parlement d'Aix           (  135,  138) MISC       Miscellaneous entities, e.g. events, nationalities, products or works of art
Charles Myriel            (  183,  185) PER        Named person or family.                 
Charles Myriel            (  259,  261) PER        Named person or family.                 
Italie     

#### we can see some wrong named entities
* *Vrai* is obviously not a location
* *Fut* is not a person
* *Myriel* is a person and not a location
*   *Baptistine* is also a person and not a location
*    *M.* it's not a person but should be used as an indicator that next token could be a person

In [7]:
def print_entities_in_sentence(spacy_doc):
    for i,sentence in enumerate(spacy_doc.sents):
        phrase = nlp(sentence.text)
        if phrase.ents:
            print(f'##### [{i:0>{3}}] ENTITY FOUND IN THIS SENTENCE :')
            displacy.render(phrase, style='ent', jupyter=True)
        else:
            print(f'##### [{i:0>{3}}] NO ENTITY FOUND IN THIS SENTENCE :{phrase.text}')
        
print_entities_in_sentence(doc)
    

##### [000] NO ENTITY FOUND IN THIS SENTENCE : Monsieur Myriel   
##### [001] ENTITY FOUND IN THIS SENTENCE :


##### [002] NO ENTITY FOUND IN THIS SENTENCE :C'était un vieillard d'environ soixante-quinze ans;
##### [003] ENTITY FOUND IN THIS SENTENCE :


##### [004] NO ENTITY FOUND IN THIS SENTENCE : Quoique ce détail ne touche en aucune manière au fond même de ce que nous avons à raconter, il n'est peut-être pas inutile, ne fût-ce que pour être exact en tout, d'indiquer ici les bruits et les propos qui avaient couru sur son compte au moment où il était arrivé dans le diocèse.
##### [005] ENTITY FOUND IN THIS SENTENCE :


##### [006] ENTITY FOUND IN THIS SENTENCE :


##### [007] NO ENTITY FOUND IN THIS SENTENCE :noblesse de robe.
##### [008] NO ENTITY FOUND IN THIS SENTENCE :On contait de lui que son père, le réservant pour hériter de sa charge, l'avait marié de fort bonne heure, à dix-huit ou vingt ans, suivant un usage assez répandu dans les familles parlementaires.
##### [009] ENTITY FOUND IN THIS SENTENCE :


...

##### [044] ENTITY FOUND IN THIS SENTENCE :


##### [045] ENTITY FOUND IN THIS SENTENCE :


##### [046] NO ENTITY FOUND IN THIS SENTENCE :elle réalisait l'idéal de ce qu'exprime le mot «respectable»;
##### [047] NO ENTITY FOUND IN THIS SENTENCE :car il semble qu'il soit nécessaire qu'une femme soit mère pour être vénérable.
##### [048] NO ENTITY FOUND IN THIS SENTENCE :Elle n'avait jamais été jolie;
##### [049] NO ENTITY FOUND IN THIS SENTENCE :toute sa vie, qui n'avait été qu'une suite de saintes oeuvres, avait fini par mettre sur elle une sorte de blancheur et de clarté;
##### [050] NO ENTITY FOUND IN THIS SENTENCE :et, en vieillissant, elle avait gagné ce qu'on pourrait appeler la beauté de la bonté.
##### [051] NO ENTITY FOUND IN THIS SENTENCE :Ce qui avait été de la maigreur dans sa jeunesse était devenu, dans sa maturité, de la transparence;
##### [052] NO ENTITY FOUND IN THIS SENTENCE :et cette diaphanéité laissait voir l'ange.
##### [053] NO ENTITY FOUND IN THIS SENTENCE :C'était une âme plus encore que ce n'était une vierge.
##### [054] NO ENTITY FOUND IN THIS SENTEN

##### [060] ENTITY FOUND IN THIS SENTENCE :


##### [061] NO ENTITY FOUND IN THIS SENTENCE :Le maire et le président lui firent la première visite, et lui de son côté fit la première visite au général et au préfet.
##### [062] NO ENTITY FOUND IN THIS SENTENCE : 
##### [063] NO ENTITY FOUND IN THIS SENTENCE :L'installation terminée, la ville attendit son évêque à l'oeuvre.
##### [064] NO ENTITY FOUND IN THIS SENTENCE :   


### Adding a Named Entity to a Span

In [8]:
# let's generate the code and store entities to remove
entities_to_remove = []
for e in doc.ents:
    if e.label_ == 'LOC' and e.text == 'Myriel':
        entities_to_remove.append(e)
        print(f'new_entities.append(Span(doc,{e.start:{3}},{e.end:{3}}, label=PER))')

new_entities.append(Span(doc,308,309, label=PER))
new_entities.append(Span(doc,455,456, label=PER))
new_entities.append(Span(doc,581,582, label=PER))
new_entities.append(Span(doc,624,625, label=PER))
new_entities.append(Span(doc,664,665, label=PER))
new_entities.append(Span(doc,678,679, label=PER))
new_entities.append(Span(doc,685,686, label=PER))
new_entities.append(Span(doc,841,842, label=PER))
new_entities.append(Span(doc,1150,1151, label=PER))


In [9]:
#let's remove the 'bad' Myriel POS entities
doc.ents = [e for e in doc.ents if e not in entities_to_remove]

In [10]:
# Get the hash value of the ORG entity label
PER = doc.vocab.strings[u'PER']  
new_entities = []
new_entities.append
# Create a Span for the new entity and add it to the list
new_entities.append(Span(doc,308,309, label=PER))
new_entities.append(Span(doc,455,456, label=PER))
new_entities.append(Span(doc,581,582, label=PER))
new_entities.append(Span(doc,624,625, label=PER))
new_entities.append(Span(doc,664,665, label=PER))
new_entities.append(Span(doc,678,679, label=PER))
# let'signore some of them on purpose
# new_entities.append(Span(doc,685,686, label=PER))
# new_entities.append(Span(doc,841,842, label=PER))
# new_entities.append(Span(doc,1150,1151, label=PER))

# Add the entity to the existing Doc object
doc.ents = list(doc.ents) + new_entities

In [11]:
for e in doc.ents:
    print(f'{e.text:{25}} {e.label_:{10}} {spacy.explain(e.label_):{40}}')

Charles                   PER        Named person or family.                 
François                  PER        Named person or family.                 
Bienvenu Myriel           PER        Named person or family.                 
Digne                     LOC        Non-GPE locations, mountain ranges, bodies of water
Digne                     LOC        Non-GPE locations, mountain ranges, bodies of water
Vrai                      LOC        Non-GPE locations, mountain ranges, bodies of water
Myriel                    PER        Named person or family.                 
parlement d'Aix           MISC       Miscellaneous entities, e.g. events, nationalities, products or works of art
Charles Myriel            PER        Named person or family.                 
Charles Myriel            PER        Named person or family.                 
Italie                    LOC        Non-GPE locations, mountain ranges, bodies of water
Myriel                    PER        Named person or family.  

there is a couple of Myriel that where not found by the end of the text

In [12]:
displacy.render(doc, style='ent', jupyter=True)

In [13]:
#but the change is not durable... as soon as you parse again the text BADABOUM
doc = nlp(chapter_1)
for e in doc.ents:
    if e.text == 'Myriel':
        print(f'{e.text:{25}} {e.label_:{10}} {spacy.explain(e.label_):{40}}')

Myriel                    PER        Named person or family.                 
Myriel                    LOC        Non-GPE locations, mountain ranges, bodies of water
Myriel                    LOC        Non-GPE locations, mountain ranges, bodies of water
Myriel                    LOC        Non-GPE locations, mountain ranges, bodies of water
Myriel                    LOC        Non-GPE locations, mountain ranges, bodies of water
Myriel                    LOC        Non-GPE locations, mountain ranges, bodies of water
Myriel                    LOC        Non-GPE locations, mountain ranges, bodies of water
Myriel                    LOC        Non-GPE locations, mountain ranges, bodies of water
Myriel                    LOC        Non-GPE locations, mountain ranges, bodies of water
Myriel                    LOC        Non-GPE locations, mountain ranges, bodies of water


### let's do it in a more durable way !

In [14]:
ruler = nlp.add_pipe("entity_ruler",before='ner')
patterns = [{"label": "PER", "pattern": "M. Myriel"},
            {"label": "PER", "pattern": [{"LOWER": "mademoiselle"}, {"LOWER": "Baptistine"}]}]
ruler.add_patterns(patterns)
doc = nlp(chapter_1)

In [15]:
for e in doc.ents:
    print(f'{e.text:{25}} {e.label_:{10}} {spacy.explain(e.label_):{40}}')

Charles                   PER        Named person or family.                 
François                  PER        Named person or family.                 
Bienvenu Myriel           PER        Named person or family.                 
Digne                     LOC        Non-GPE locations, mountain ranges, bodies of water
Digne                     LOC        Non-GPE locations, mountain ranges, bodies of water
Vrai                      LOC        Non-GPE locations, mountain ranges, bodies of water
M. Myriel                 PER        Named person or family.                 
parlement d'Aix           MISC       Miscellaneous entities, e.g. events, nationalities, products or works of art
Charles Myriel            PER        Named person or family.                 
Charles Myriel            PER        Named person or family.                 
Italie                    LOC        Non-GPE locations, mountain ranges, bodies of water
M. Myriel                 PER        Named person or family.  

In [17]:
print_entities_in_sentence(doc)

##### [000] NO ENTITY FOUND IN THIS SENTENCE : Monsieur Myriel   
##### [001] ENTITY FOUND IN THIS SENTENCE :


##### [002] NO ENTITY FOUND IN THIS SENTENCE :C'était un vieillard d'environ soixante-quinze ans;
##### [003] ENTITY FOUND IN THIS SENTENCE :


##### [004] NO ENTITY FOUND IN THIS SENTENCE : Quoique ce détail ne touche en aucune manière au fond même de ce que nous avons à raconter, il n'est peut-être pas inutile, ne fût-ce que pour être exact en tout, d'indiquer ici les bruits et les propos qui avaient couru sur son compte au moment où il était arrivé dans le diocèse.
##### [005] ENTITY FOUND IN THIS SENTENCE :


##### [006] ENTITY FOUND IN THIS SENTENCE :


##### [007] NO ENTITY FOUND IN THIS SENTENCE :noblesse de robe.
##### [008] NO ENTITY FOUND IN THIS SENTENCE :On contait de lui que son père, le réservant pour hériter de sa charge, l'avait marié de fort bonne heure, à dix-huit ou vingt ans, suivant un usage assez répandu dans les familles parlementaires.
##### [009] ENTITY FOUND IN THIS SENTENCE :


...

##### [044] ENTITY FOUND IN THIS SENTENCE :


##### [045] ENTITY FOUND IN THIS SENTENCE :


##### [046] NO ENTITY FOUND IN THIS SENTENCE :elle réalisait l'idéal de ce qu'exprime le mot «respectable»;
##### [047] NO ENTITY FOUND IN THIS SENTENCE :car il semble qu'il soit nécessaire qu'une femme soit mère pour être vénérable.
##### [048] NO ENTITY FOUND IN THIS SENTENCE :Elle n'avait jamais été jolie;
##### [049] NO ENTITY FOUND IN THIS SENTENCE :toute sa vie, qui n'avait été qu'une suite de saintes oeuvres, avait fini par mettre sur elle une sorte de blancheur et de clarté;
##### [050] NO ENTITY FOUND IN THIS SENTENCE :et, en vieillissant, elle avait gagné ce qu'on pourrait appeler la beauté de la bonté.
##### [051] NO ENTITY FOUND IN THIS SENTENCE :Ce qui avait été de la maigreur dans sa jeunesse était devenu, dans sa maturité, de la transparence;
##### [052] NO ENTITY FOUND IN THIS SENTENCE :et cette diaphanéité laissait voir l'ange.
##### [053] NO ENTITY FOUND IN THIS SENTENCE :C'était une âme plus encore que ce n'était une vierge.
##### [054] NO ENTITY FOUND IN THIS SENTEN

##### [060] ENTITY FOUND IN THIS SENTENCE :


##### [061] NO ENTITY FOUND IN THIS SENTENCE :Le maire et le président lui firent la première visite, et lui de son côté fit la première visite au général et au préfet.
##### [062] NO ENTITY FOUND IN THIS SENTENCE : 
##### [063] NO ENTITY FOUND IN THIS SENTENCE :L'installation terminée, la ville attendit son évêque à l'oeuvre.
##### [064] NO ENTITY FOUND IN THIS SENTENCE :   


### Much better but,  what if we used the large model as is ?

In [18]:
# load the big fat french model
# https://spacy.io/models/fr#fr_core_news_lg
nlp_large = spacy.load('fr_core_news_lg')


In [19]:
doc_large = nlp_large(chapter_1)

In [21]:
for e in doc_large.ents:
    print(f'{e.text:{25}} {e.label_:{10}} {spacy.explain(e.label_):{40}}')

Myriel                    PER        Named person or family.                 
M. Charles-François-Bienvenu Myriel PER        Named person or family.                 
Digne                     LOC        Non-GPE locations, mountain ranges, bodies of water
Digne                     LOC        Non-GPE locations, mountain ranges, bodies of water
Myriel                    PER        Named person or family.                 
Charles Myriel            PER        Named person or family.                 
Charles Myriel            PER        Named person or family.                 
Italie                    LOC        Non-GPE locations, mountain ranges, bodies of water
M. Myriel                 PER        Named person or family.                 
Fut                       PER        Named person or family.                 
d'Italie                  LOC        Non-GPE locations, mountain ranges, bodies of water
M. Myriel                 PER        Named person or family.                 
Brignolles

In [22]:
print_entities_in_sentence(doc_large)

##### [000] NO ENTITY FOUND IN THIS SENTENCE : 
##### [001] ENTITY FOUND IN THIS SENTENCE :


##### [002] ENTITY FOUND IN THIS SENTENCE :


##### [003] NO ENTITY FOUND IN THIS SENTENCE :C'était un vieillard d'environ soixante-quinze ans;
##### [004] ENTITY FOUND IN THIS SENTENCE :


##### [005] NO ENTITY FOUND IN THIS SENTENCE : Quoique ce détail ne touche en aucune manière au fond même de ce que nous avons à raconter, il n'est peut-être pas inutile, ne fût-ce que pour être exact en tout, d'indiquer ici les bruits et les propos qui avaient couru sur son compte au moment où il était arrivé dans le diocèse.
##### [006] ENTITY FOUND IN THIS SENTENCE :


##### [007] ENTITY FOUND IN THIS SENTENCE :


...

##### [048] ENTITY FOUND IN THIS SENTENCE :


##### [049] ENTITY FOUND IN THIS SENTENCE :


##### [050] NO ENTITY FOUND IN THIS SENTENCE :elle réalisait l'idéal de ce qu'exprime le mot «respectable»;
##### [051] NO ENTITY FOUND IN THIS SENTENCE :car il semble qu'il soit nécessaire qu'une femme soit mère pour être vénérable.
##### [052] NO ENTITY FOUND IN THIS SENTENCE :Elle n'avait jamais été jolie;
##### [053] NO ENTITY FOUND IN THIS SENTENCE :toute sa vie, qui n'avait été qu'une suite de saintes oeuvres, avait fini par mettre sur elle une sorte de blancheur et de clarté; et, en vieillissant, elle avait gagné ce qu'on pourrait appeler la beauté de la bonté.
##### [054] NO ENTITY FOUND IN THIS SENTENCE :Ce qui avait été de la maigreur dans sa jeunesse était devenu, dans sa maturité, de la transparence;
##### [055] NO ENTITY FOUND IN THIS SENTENCE :et cette diaphanéité laissait voir l'ange.
##### [056] NO ENTITY FOUND IN THIS SENTENCE :C'était une âme plus encore que ce n'était une vierge.
##### [057] NO ENTITY FOUND IN THIS SENTENCE :Sa personne semblait faite d'ombre;
##### 

##### [065] ENTITY FOUND IN THIS SENTENCE :


##### [066] NO ENTITY FOUND IN THIS SENTENCE :Le maire et le président lui firent la première visite, et lui de son côté fit la première visite au général et au préfet.
##### [067] NO ENTITY FOUND IN THIS SENTENCE : 
##### [068] NO ENTITY FOUND IN THIS SENTENCE :L'installation terminée, la ville attendit son évêque à l'oeuvre.
##### [069] NO ENTITY FOUND IN THIS SENTENCE :   
